In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

## Model

In [26]:
def join(series):
    #return ' '.join( str(list(series)).strip('[]').split(', ') )
    return ' '.join( map(str, series) )

In [27]:
%run 'metrics.ipynb'
def model_evaluation(actual, prediction):
    return mapk(actual['ad_id'], prediction['ad_id'], k=12)

In [3]:
pd.read_csv("../generated/final/clicks_train_sample.csv", nrows=10).head()

,display_id,document_id,timestamp,traffic_source,platform,day,hour,geo,ad_id,clicked,adsPerDisplay,clicksPerShows
0,8,1330329,638,1.0,2.0,1,7,2765,95724,0,4,0.034785
1,8,1330329,638,1.0,2.0,1,7,2765,175694,0,4,0.202327
2,8,1330329,638,1.0,2.0,1,7,2765,280430,1,4,0.256853
3,8,1330329,638,1.0,2.0,1,7,2765,329774,0,4,0.052426
4,2657,1271490,185255,1.0,2.0,1,7,2765,70081,0,4,0.150145


In [4]:
group_by=['ad_id', 'adsPerDisplay']
usecols = ['display_id', 'ad_id', 'adsPerDisplay', 'clicked']
alpha=10
beta=0.1
bottom=30

## Validation

### Step 1

In [ ]:
clicks = pd.read_csv("../generated/final/clicks_train_sample.csv", usecols = usecols)
clicks.count()

In [ ]:
res = pd.DataFrame({'total_viewed': clicks.groupby(['ad_id'])['clicked'].size() }).reset_index()
res.count()

In [ ]:
clicks = clicks.merge(res, on=['ad_id'])
del res
clicks.head()

In [ ]:
res = pd.DataFrame({'group_viewed':clicks.groupby(group_by)['clicked'].size()}).reset_index()
res.head()

In [ ]:
clicks = clicks.merge(res, on=group_by)
del res
clicks.head()

In [ ]:
clicks.to_csv("../generated/solutions/8_clicks_train_sample.csv", index=False)

### Step 2

In [ ]:
clicks = pd.read_csv("../generated/solutions/8_clicks_train_sample.csv")

In [ ]:
res = pd.DataFrame({'group_size':clicks.groupby('adsPerDisplay')['clicked'].size()}).reset_index()
total_size = len(clicks['clicked'])
res['group_bottom'] = np.rint(res['group_size'] / total_size * bottom).astype(int)
res.drop(['group_size'], axis=1, inplace=True)
res.head(12)

In [ ]:
clicks = clicks.merge(res, on=['adsPerDisplay'])
clicks.head()

In [ ]:
clicks.to_csv("../generated/solutions/8_clicks_train_sample.b20.csv", index=False)

In [ ]:
clicksPerAd_train = pd.DataFrame({'group_clicked': clicks.groupby(group_by)['clicked'].sum(),
                                   'group_viewed': clicks.groupby(group_by)['clicked'].size()
                   }).reset_index()

clicksPerAd_train = clicksPerAd_train.merge(res, on='adsPerDisplay')
clicksPerAd_train.count()

In [ ]:
clicksPerAd_train.head()

In [ ]:
clicksPerAd_train.to_csv("../generated/solutions/8_clicksPerAd_train.b20.csv", index=False)

### Step 3

In [5]:
clicks = pd.read_csv("../generated/solutions/8_clicks_train_sample.b30.csv")
clicks.head()

,display_id,ad_id,clicked,adsPerDisplay,total_viewed,group_viewed,group_bottom
0,8,95724,0,4,8337,3757,7
1,280348,95724,0,4,8337,3757,7
2,3960762,95724,0,4,8337,3757,7
3,7315944,95724,0,4,8337,3757,7
4,7492181,95724,0,4,8337,3757,7


In [6]:
clicks.count()

display_id       65355241
ad_id            65355241
clicked          65355241
adsPerDisplay    65355241
total_viewed     65355241
group_viewed     65355241
group_bottom     65355241
dtype: int64

In [7]:
clicksPerAd_train = pd.read_csv("../generated/solutions/8_clicksPerAd_train.b30.csv")
clicksPerAd_train.head()

,ad_id,adsPerDisplay,group_clicked,group_viewed,group_bottom
0,1,3,0,1,2
1,3,3,0,1,2
2,4,3,0,3,2
3,7,3,3,22,2
4,17,3,0,1,2


In [10]:
clicksPerAd_train_means = pd.DataFrame({'group_mean':clicks.groupby('adsPerDisplay')['clicked'].mean()
                                      #'viewed':clicks.groupby('adsPerDisplay')['clicked'].size() #???                                          
                                        }).reset_index()

In [11]:
clicksPerAd_train_means.head(20)

,adsPerDisplay,group_mean
0,2,0.500000
1,3,0.333333
2,4,0.250000
3,5,0.200000
4,6,0.166667
5,7,0.142857
6,8,0.125000
7,9,0.111111
8,10,0.100000
9,11,0.090909


In [12]:
clicksPerAd_train = clicksPerAd_train.merge(clicksPerAd_train_means, on='adsPerDisplay')
clicksPerAd_train.head()

,ad_id,adsPerDisplay,group_clicked,group_viewed,group_bottom,group_mean
0,1,3,0,1,2,0.333333
1,3,3,0,1,2,0.333333
2,4,3,0,3,2,0.333333
3,7,3,3,22,2,0.333333
4,17,3,0,1,2,0.333333


In [13]:
res_mean = clicks['clicked'].mean()              #clicks[clicks.total_viewed>bottom]['clicked'].mean()
res_mean

0.19362743991717513

In [14]:
clicksPerAd_train['clicksPerShows'] = (beta*res_mean + alpha*clicksPerAd_train['group_mean'] + clicksPerAd_train['group_clicked']) / (beta + alpha + clicksPerAd_train['group_viewed'])
#clicksPerAd_train['clicksPerShows'] = clicksPerAd_train['group_clicked'] / clicksPerAd_train['group_viewed']

clicksPerAd_train.head()

,ad_id,adsPerDisplay,group_clicked,group_viewed,group_bottom,group_mean,clicksPerShows
0,1,3,0,1,2,0.333333,0.302045
1,3,3,0,1,2,0.333333,0.302045
2,4,3,0,3,2,0.333333,0.255931
3,7,3,3,22,2,0.333333,0.197903
4,17,3,0,1,2,0.333333,0.302045


In [15]:
clicksPerAd_train.drop(['group_clicked', 'group_mean'], axis=1, inplace=True)

In [16]:
clicksPerAd_train.count()

ad_id             1406219
adsPerDisplay     1406219
group_viewed      1406219
group_bottom      1406219
clicksPerShows    1406219
dtype: int64

In [17]:
clicksPerAd_train_filtered = clicksPerAd_train[clicksPerAd_train['group_viewed']>clicksPerAd_train['group_bottom']].copy()
clicksPerAd_train_filtered.count()

ad_id             506947
adsPerDisplay     506947
group_viewed      506947
group_bottom      506947
clicksPerShows    506947
dtype: int64

In [18]:
test = pd.read_csv("../generated/final/clicks_test_sample.csv", usecols = usecols)
test.count()

display_id       21786490
ad_id            21786490
clicked          21786490
adsPerDisplay    21786490
dtype: int64

In [19]:
validation = test[test['clicked']==1].astype({'ad_id':str}).sort_values('display_id')
validation.count()

display_id       4220025
ad_id            4220025
clicked          4220025
adsPerDisplay    4220025
dtype: int64

In [20]:
validation.head()

,display_id,ad_id,clicked,adsPerDisplay
18142714,1,144739,1,6
14412118,7,105766,1,3
11270842,9,140940,1,4
377269,14,224171,1,4
18233315,26,152193,1,2


In [21]:
on=['ad_id', 'adsPerDisplay']
clicksPerAd_train_filtered.head()

,ad_id,adsPerDisplay,group_viewed,group_bottom,clicksPerShows
2,4,3,3,2,0.255931
3,7,3,22,2,0.197903
13,44,3,6,2,0.208242
14,45,3,9,2,0.175534
15,48,3,4,2,0.379624


In [22]:
prediction = pd.merge(test, clicksPerAd_train_filtered, how='left', on = ['ad_id', 'adsPerDisplay']) 
prediction.count()

display_id        21786490
ad_id             21786490
clicked           21786490
adsPerDisplay     21786490
group_viewed      21089900
group_bottom      21089900
clicksPerShows    21089900
dtype: int64

In [23]:
clicksPerAd_train_means = pd.DataFrame({'clicksPerShows':clicksPerAd_train.groupby('adsPerDisplay')['clicksPerShows'].mean()}).reset_index()
clicksPerAd_train_means.head(20)

,adsPerDisplay,clicksPerShows
0,2,0.483844
1,3,0.323173
2,4,0.238194
3,5,0.191648
4,6,0.158580
5,7,0.141108
6,8,0.118518
7,9,0.102351
8,10,0.089511
9,11,0.091472


In [24]:
index = pd.isnull(prediction['clicksPerShows'])
prediction_nulls = prediction[index].copy()
prediction_nulls.drop(['clicksPerShows'], axis=1, inplace=True)
prediction_nulls = prediction_nulls.merge(clicksPerAd_train_means, how='left', on = 'adsPerDisplay')#.fillna(clicksPerAd_train['clicksPerShows'].mean())
prediction = pd.concat([prediction[~index], prediction_nulls])
    
prediction = prediction.sort_values(by=['display_id', 'clicksPerShows'], ascending=[True, False])
print len(prediction[prediction['clicksPerShows'].isnull()])  

0


In [28]:
prediction = pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).reset_index()
prediction.count()

display_id    4220025
ad_id         4220025
dtype: int64

In [29]:
print 'clicksPerShows :', model_evaluation(validation, prediction)  # 0.649366311611

clicksPerShows : 0.651457961706


## Training

## Predicting

Check order:

## Submission